# Into to Keras

based on http://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

## Load Data

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
seed = 23
np.random.seed(seed)

Using Theano backend.


Let's get some data on wine quality
wget http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv

In [2]:
# load wine quality dataset
dataset = np.loadtxt("winequality-white.csv", delimiter=";", skiprows=1)
N, K = dataset.shape
print(N, K)
dataset[0,11]

(4898, 12)


6.0

In [3]:
# split into input (X) and output (Y) variables
X = dataset[:,0:K-1]
Y = dataset[:,K-1]

In [4]:
print(len(dataset[0]))
X[0]

12


array([  7.00000000e+00,   2.70000000e-01,   3.60000000e-01,
         2.07000000e+01,   4.50000000e-02,   4.50000000e+01,
         1.70000000e+02,   1.00100000e+00,   3.00000000e+00,
         4.50000000e-01,   8.80000000e+00])

In [5]:
import pandas
df = pandas.DataFrame(Y, columns=["y"])
df['y'] = df['y'].astype('category')

In [6]:
df.describe()

,y
count,4898.0
unique,7.0
top,6.0
freq,2198.0


In [7]:
#Let's simplify this problem for a moment 
Y2 = [0 if y <= 6 else 1 for y in Y]
print(Y2[0:20])
df2 = pandas.DataFrame(Y2, columns=["y"])
df2.describe()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0]


,y
count,4898.000000
mean,0.216415
std,0.411842
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


## Define and Compite the Model

Many models in Keras are defined as a sequence of layers. Let's create a logistic regression model. Here input_dim is the dimension of your data. init='uniform' tells Keras to initialize the weights with a uniform distribution (in this case from numbers from 0 to 0.05). 

In [8]:
# create model
model = Sequential()
model.add(Dense(1, input_dim=K-1, init='uniform', activation='sigmoid'))

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 1)             12          dense_input_1[0][0]              
Total params: 12
____________________________________________________________________________________________________


For compiling the model we should specify the loss function, the gradient descent algorithm. We can also add metrics such as the 'accuracy' to collect and report.

In [10]:
# Compile model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## Fit and Evaluate the Model 

**nb_epoch** is the number of iteration performed over the data

**batch_size** is the number of instances that are evaluated before a weight update.

In [11]:
# model.fit(X, Y2, nb_epoch=150, batch_size=50)

In [12]:
# let's keep part of our data as validation
np.random.shuffle(dataset)

In [13]:
dataset_valid = dataset[:500,:]
dataset_test = dataset[500:1000,:]
dataset_train = dataset[1000:,:]

In [14]:
dataset_valid.shape

(500, 12)

In [15]:
def get_X_Y(dataset):
    X = dataset[:,0:K-1]
    Y = dataset[:,K-1]
    Y2 = [0 if y <= 6 else 1 for y in Y]
    return X, Y2

In [16]:
X_train, Y_train = get_X_Y(dataset_train)
X_valid, Y_valid = get_X_Y(dataset_valid)
X_test, Y_test = get_X_Y(dataset_test)

In [17]:
# scaling data
# It is recommended to scale inputs
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
X_Train = scaler.fit_transform(X_train)
X_Valid = scaler.transform(X_valid)
X_Test = scaler.transform(X_test)


In [18]:
print(X_Train[0])
print(X_train[0])

[ 1.23140095  3.55082944  1.0552557   0.83748605  9.21178345 -0.11855874
  2.11556993  1.42591267 -2.11237415  2.18845605 -1.15563926]
[   7.9       0.64      0.46     10.6       0.244    33.      227.
    0.9983    2.87      0.74      9.1   ]


In [19]:
model.fit(X_Train, Y_train, nb_epoch=50, batch_size=50, validation_data=[X_Valid, Y_valid])

Train on 3898 samples, validate on 500 samples
Epoch 1/50
3898/3898 [==============================] - 0s - loss: 0.6725 - acc: 0.6342 - val_loss: 0.6518 - val_acc: 0.6600
Epoch 2/50
3898/3898 [==============================] - 0s - loss: 0.6353 - acc: 0.7011 - val_loss: 0.6228 - val_acc: 0.7020
Epoch 3/50
3898/3898 [==============================] - 0s - loss: 0.6082 - acc: 0.7304 - val_loss: 0.6006 - val_acc: 0.7240
Epoch 4/50
3898/3898 [==============================] - 0s - loss: 0.5862 - acc: 0.7468 - val_loss: 0.5816 - val_acc: 0.7480
Epoch 5/50
3898/3898 [==============================] - 0s - loss: 0.5674 - acc: 0.7581 - val_loss: 0.5656 - val_acc: 0.7560
Epoch 6/50
3898/3898 [==============================] - 0s - loss: 0.5511 - acc: 0.7727 - val_loss: 0.5511 - val_acc: 0.7680
Epoch 7/50
3898/3898 [==============================] - 0s - loss: 0.5366 - acc: 0.7812 - val_loss: 0.5385 - val_acc: 0.7640
Epoch 8/50
3898/3898 [==============================] - 0s - loss: 0.5238 - ac

In [20]:
model.get_weights()

[array([[ 0.12483837],
        [-0.32502642],
        [-0.05510116],
        [ 0.53180546],
        [-0.31839779],
        [ 0.19786701],
        [-0.13290852],
        [-0.46295828],
        [ 0.23405541],
        [ 0.17872934],
        [ 0.80128884]], dtype=float32), array([-1.60823309], dtype=float32)]

In [21]:
model.evaluate(X_Test, Y_test)

 32/500 [>.............................] - ETA: 0s

[0.42596645832061769, 0.8080000009536743]

In [22]:
Yhat = model.predict(X_Test) ## these are probabilities
Yhat_class = model.predict_classes(X_Test)

 32/500 [>.............................] - ETA: 0s

In [23]:
#Yhat_class
#Yhat

## First Neural Network

In [24]:
# create model with a hidden layer
model2 = Sequential()
model2.add(Dense(20, input_dim=K-1, init='uniform', activation='relu'))
model2.add(Dense(1, init='uniform', activation='sigmoid'))

In [25]:
model2.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_2 (Dense)                  (None, 20)            240         dense_input_2[0][0]              
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 1)             21          dense_2[0][0]                    
Total params: 261
____________________________________________________________________________________________________


In [26]:
model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
model2.optimizer.lr.get_value()
#model2.optimizer.lr.set_value(0.0001)
#model2.optimizer.lr.get_value()

array(0.0010000000474974513, dtype=float32)

In [28]:
model.optimizer.lr.set_value(0.01)

In [29]:
model2.fit(X_Train, Y_train, nb_epoch=30, batch_size=50, validation_data=[X_Valid, Y_valid])

Train on 3898 samples, validate on 500 samples
Epoch 1/30
3898/3898 [==============================] - 0s - loss: 0.6451 - acc: 0.7819 - val_loss: 0.5787 - val_acc: 0.7740
Epoch 2/30
3898/3898 [==============================] - 0s - loss: 0.5165 - acc: 0.7842 - val_loss: 0.4882 - val_acc: 0.7740
Epoch 3/30
3898/3898 [==============================] - 0s - loss: 0.4503 - acc: 0.7858 - val_loss: 0.4542 - val_acc: 0.7840
Epoch 4/30
3898/3898 [==============================] - 0s - loss: 0.4239 - acc: 0.8073 - val_loss: 0.4390 - val_acc: 0.7880
Epoch 5/30
3898/3898 [==============================] - 0s - loss: 0.4113 - acc: 0.8163 - val_loss: 0.4302 - val_acc: 0.7940
Epoch 6/30
3898/3898 [==============================] - 0s - loss: 0.4046 - acc: 0.8153 - val_loss: 0.4266 - val_acc: 0.7980
Epoch 7/30
3898/3898 [==============================] - 0s - loss: 0.4001 - acc: 0.8166 - val_loss: 0.4251 - val_acc: 0.7960
Epoch 8/30
3898/3898 [==============================] - 0s - loss: 0.3972 - ac

In [30]:
model.optimizer.lr.set_value(0.001)

In [31]:
model2.fit(X_Train, Y_train, nb_epoch=30, batch_size=50, validation_data=[X_Valid, Y_valid])

Train on 3898 samples, validate on 500 samples
Epoch 1/30
3898/3898 [==============================] - 0s - loss: 0.3763 - acc: 0.8207 - val_loss: 0.4126 - val_acc: 0.7980
Epoch 2/30
3898/3898 [==============================] - 0s - loss: 0.3759 - acc: 0.8217 - val_loss: 0.4117 - val_acc: 0.8000
Epoch 3/30
3898/3898 [==============================] - 0s - loss: 0.3748 - acc: 0.8225 - val_loss: 0.4127 - val_acc: 0.8040
Epoch 4/30
3898/3898 [==============================] - 0s - loss: 0.3751 - acc: 0.8207 - val_loss: 0.4158 - val_acc: 0.8060
Epoch 5/30
3898/3898 [==============================] - 0s - loss: 0.3740 - acc: 0.8207 - val_loss: 0.4129 - val_acc: 0.8060
Epoch 6/30
3898/3898 [==============================] - 0s - loss: 0.3735 - acc: 0.8222 - val_loss: 0.4123 - val_acc: 0.8060
Epoch 7/30
3898/3898 [==============================] - 0s - loss: 0.3729 - acc: 0.8220 - val_loss: 0.4126 - val_acc: 0.8060
Epoch 8/30
3898/3898 [==============================] - 0s - loss: 0.3721 - ac

In [32]:
model.optimizer.lr.set_value(0.0001)

In [33]:
model2.fit(X_Train, Y_train, nb_epoch=50, batch_size=50, validation_data=[X_Valid, Y_valid])

Train on 3898 samples, validate on 500 samples
Epoch 1/50
3898/3898 [==============================] - 0s - loss: 0.3638 - acc: 0.8240 - val_loss: 0.4152 - val_acc: 0.8080
Epoch 2/50
3898/3898 [==============================] - 0s - loss: 0.3637 - acc: 0.8240 - val_loss: 0.4163 - val_acc: 0.8060
Epoch 3/50
3898/3898 [==============================] - 0s - loss: 0.3633 - acc: 0.8258 - val_loss: 0.4161 - val_acc: 0.8080
Epoch 4/50
3898/3898 [==============================] - 0s - loss: 0.3629 - acc: 0.8256 - val_loss: 0.4149 - val_acc: 0.8120
Epoch 5/50
3898/3898 [==============================] - 0s - loss: 0.3633 - acc: 0.8253 - val_loss: 0.4171 - val_acc: 0.8080
Epoch 6/50
3898/3898 [==============================] - 0s - loss: 0.3629 - acc: 0.8250 - val_loss: 0.4173 - val_acc: 0.8100
Epoch 7/50
3898/3898 [==============================] - 0s - loss: 0.3621 - acc: 0.8258 - val_loss: 0.4175 - val_acc: 0.8080
Epoch 8/50
3898/3898 [==============================] - 0s - loss: 0.3625 - ac

In [34]:
model2.evaluate(X_Test, Y_test)

 32/500 [>.............................] - ETA: 0s

[0.3660741767883301, 0.84000000095367433]

## Ensemble

In [35]:
model3 = Sequential()
model3.add(Dense(1, input_dim=2, init='uniform', activation='sigmoid'))

In [36]:
# getting predictions from the two models
m1 = model.predict(X_Train)
m2 = model2.predict(X_Train)
x = np.column_stack((m1, m2))

In [37]:
print(m1[0], m2[0], Y_train[0])

(array([ 0.00078547], dtype=float32), array([  3.62538941e-08], dtype=float32), 0)


In [38]:
x[0]

array([  7.85468670e-04,   3.62538941e-08], dtype=float32)

In [39]:
model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model3.fit(x, Y_train, nb_epoch=50, batch_size=50)

Epoch 1/50
3898/3898 [==============================] - 0s - loss: 0.6827 - acc: 0.7676     
Epoch 2/50
3898/3898 [==============================] - 0s - loss: 0.6621 - acc: 0.7842     
Epoch 3/50
3898/3898 [==============================] - 0s - loss: 0.6441 - acc: 0.7842     
Epoch 4/50
3898/3898 [==============================] - 0s - loss: 0.6282 - acc: 0.7842     
Epoch 5/50
3898/3898 [==============================] - 0s - loss: 0.6141 - acc: 0.7842     
Epoch 6/50
3898/3898 [==============================] - 0s - loss: 0.6014 - acc: 0.7842     
Epoch 7/50
3898/3898 [==============================] - 0s - loss: 0.5898 - acc: 0.7842     
Epoch 8/50
3898/3898 [==============================] - 0s - loss: 0.5792 - acc: 0.7842     
Epoch 9/50
3898/3898 [==============================] - 0s - loss: 0.5694 - acc: 0.7842     
Epoch 10/50
3898/3898 [==============================] - 0s - loss: 0.5602 - acc: 0.7842     
Epoch 11/50
3898/3898 [==============================] - 0s - loss: 0

In [40]:
model3.get_weights()

[array([[ 0.9050945 ],
        [ 2.56203246]], dtype=float32), array([-2.03053474], dtype=float32)]

In [41]:
m1_test = model.predict(X_Test)
m2_test = model2.predict(X_Test)
x_test = np.column_stack((m1_test, m2_test))

In [42]:
model3.evaluate(x_test, Y_test)

 32/500 [>.............................] - ETA: 0s

[0.40323701620101926, 0.83200000095367432]